This notebook can be used to generate additional images for self-supervised learning. It crops similar bounding boxes on the images before and after the ones given with damages to have higher diversity in the dataset for self supervised learning.

In [16]:
import json
import PIL
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import pandas
from pathlib import Path
import os

The steps:
1. Get a list of the ids
2. Get a list of image id's pointing to images ls


Make sure your cwd is in /notebooks/preprocessing/. 

In [17]:
file = open(Path("../original.json"))
original = json.load(file)
path = Path("../../../data/cropped_additional/")
try:
    path.mkdir(parents=True, exist_ok=False)
except FileExistsError:
    print("Folder is already there")
else:
    print("Folder was created")

Folder is already there


In [18]:
id_list = []
same_images = []

for entry in original["annotations"]:
    if entry['id'] not in id_list:
        id_list.append(entry['id'])

#populate ph with image id's that has more than 1 image connected to it!
ph =[]
for entry in original["annotations"]:
    for entry2 in original["annotations"]:
        if entry2['image_id'] == entry['image_id'] and entry['id'] != entry2['id'] and entry["image_id"] not in ph:
            ph.append(entry2['image_id'])
        
for _ in ph:
    tmp = [_]
    same_images.append(tmp)
    

for entry in original["annotations"]:
    im_id = entry['image_id']
    id = entry['id']
    for list in same_images:
        if im_id in list and id not in list:
            list.append(id)


print(r"Length of image id's {}".format(len(same_images)))

Length of image id's 130


In [19]:
def crop_region(image: np.array, bounding_box) -> np.array:
    """

    Args:
        image: numpy type
        bounding_box: Integer

    Returns: numpy type

    """
    x_min, y_min, x_range, y_range = bounding_box

    _ = np.flipud(image[x_min:x_min + x_range, y_min:y_min + y_range])
    _ = np.rot90(_,k=3)

    return _


In [20]:
image_root = Path("..", "..", "..", "data", "Images")


1. Go through all the images in the same_images list
2. Recrop, if the name already exists give a suffix with the image_id

In [21]:
#Unpack to tuple for ease of use

overlap = []
for list in same_images:
    for item in list:
        # First element is image_id
        if item == list[0]:
            continue
        ################################
        # From here it'll always take id from the list
        overlap.append((list[0],item))

print(r"Count of overlap set is {}".format(len(overlap)))

#After cropping this number should match the number of entries in overlap.json

Count of overlap set is 295


In [22]:
non_overlap = []
for entry in original["annotations"]:
    if (entry["image_id"],entry["id"]) not in overlap:
        non_overlap.append((entry["image_id"], entry["id"]))
print(len(non_overlap))
print(len(overlap) + len(non_overlap))
if(len(overlap) + len(non_overlap) == len(original["annotations"])):
    print("Count of disjunct set matches original json entries")


602
897
Count of disjunct set matches original json entries


In [23]:
# Set up new json
# open JSON file
categories = original['categories']
images = original['images']
annotations = original['annotations']

# key image ids as key for dict
images_keys = [img['id'] for img in images]

# get dictionary for filenames
images_dict = dict(zip(images_keys, images))
# creat a new json file to store the new infomation

print(len(categories))
print(len(images))
print(len(annotations))


1
9585
897


In [24]:
newfile = {
    "categories": [{
        "id": 0,
        "name": "dent"
    }, {
        "id": 1,
        "name": "other"
    }, {
        "id": 2,
        "name": "rim"
    }, {
        "id": 3,
        "name": "scratch"
    }, {
        "id": 4,
        "name": "deformation"
    }],
    "images": [],
    "annotations": []
}

# Deal with non overlapping ones first

# take prelabeled labels and put them by image_id(because they are non overlap and unique)
#from 600-896 (1 to 897 in total, index from 0)
pre_den = json.load(open(Path("..","..", "etc", "prelabeled_den.json")))
#from 300-599
pre_mag = json.load(open(Path("..", "..", "etc", "prelabeled_mag.json")))
#from 0-299
#pre_thu = json.load(open(Path("..", "etc", "prelabeled_thu.json")))
stitched = {"annotations": []}

for i in range(300):
    stitched["annotations"].append(pre_mag["annotations"][i])

for i in range(300,600):
    stitched["annotations"].append(pre_mag["annotations"][i])

for i in range(600,897):
    stitched["annotations"].append(pre_den["annotations"][i])

print(len(stitched["annotations"]))


897


In [25]:
count =0
relabeled_count=0
for newentry in stitched["annotations"]:
    for entry in original["annotations"]:
        if(entry["id"] == newentry["id"]):
            count+=1
            if(entry["category_id"] != newentry["category_id"]):
                relabeled_count+=1
if count == len(original["annotations"]):
    print(r"{} entries matched, valid".format(count))

print(r"{} entries were relabeled.".format(relabeled_count))

897 entries matched, valid
725 entries were relabeled.


In [26]:
problem_set = []

def get_labeled_category(id,img_id):
    for entry in stitched["annotations"]:
        if entry["id"] == id and entry["image_id"] == img_id:
            return entry["category_id"]
    print("Not found!")
    problem_set.append((id, img_id))


images_entries = original["images"]
cropped_path = Path("..", "..", "..", "data", "cropped_additional")
if (len(problem_set) == 0):
    print("No problems found.")


No problems found.


In [28]:
#only need the annotations entries since the rest are obsolete

not_found = 0

for entry in original["annotations"]:
    if((entry["image_id"],entry["id"]) in non_overlap ):
        # Only access images in the list
        id = entry["id"]
        image_id = entry['image_id']
        bounding_box = entry['bbox']
        for i in [2,3]:
            if bounding_box[i] < 150:
                if bounding_box[i-2]-75 >0:
                    bounding_box[i - 2] = bounding_box[i - 2] - 75
                    bounding_box[i] = 150
                else:
                    bounding_box[i] = 150
        filename = images_dict[image_id]['file_name']
        #get filename of file before given image
        [filename_begin, _] = filename.split('.')
        for i in range(1,5):
            filename_split = filename_begin.split('_')
            filename_split[-1] = str(int(filename_split[-1]) + i)
            filename_begin = '_'.join(filename_split)
            if Path(image_root,'.'.join([filename_begin, 'webp'])).is_file():
                filename = '.'.join([filename_begin, 'webp'])
                break
            elif Path(image_root,'.'.join([filename_begin, 'jpeg'])).is_file():
                filename = '.'.join([filename_begin, 'jpeg'])
                break
            elif i==5:
                print(filename)
                filename = None
        if filename:
            fp = Path(image_root, filename)
            #ONLY get the category_id from prelabeled ones for NON overlap, overlap ones need relabeling, and shall be marked with category_id 1
            category_id = get_labeled_category(id,image_id)
            img = Image.open(fp)
            numpy_image = np.transpose(np.array(img), (1, 0, 2))
            cropped_image = crop_region(numpy_image, bounding_box)
            cropped_image = Image.fromarray(cropped_image)
            save_path = Path(cropped_path,str(id)+"+1.png")
            cropped_image.save(save_path)
            save_path = save_path.relative_to("../../../data/").as_posix()
            fp = fp.relative_to("../../../data").as_posix()
            # Fill out the data
            # image id refers to the source, non cropped image
            new_annotation = {
                "id": id,
                "category_id": category_id,
                'image_id': image_id,
                'filepath': save_path,
                'source': fp,
                "bbox": bounding_box
            }

            newfile['annotations'].append(new_annotation)
        else:
            not_found += 1

print("not_found: " + str(not_found))


not_found: 0


In [29]:
#only need the annotations entries since the rest are obsolete

not_found = 0

for entry in original["annotations"]:
    if((entry["image_id"],entry["id"]) in non_overlap ):
        # Only access images in the list
        id = entry["id"]
        image_id = entry['image_id']
        bounding_box = entry['bbox']
        for i in [2, 3]:
            if bounding_box[i] < 150:
                if bounding_box[i - 2] - 75 > 0:
                    bounding_box[i - 2] = bounding_box[i - 2] - 75
                    bounding_box[i] = 150
                else:
                    bounding_box[i] = 150
        filename = images_dict[image_id]['file_name']
        #print(filename)
        #get filename of file before given image
        [filename_begin, _] = filename.split('.')
        for i in range(1,5):
            filename_split = filename_begin.split('_')
            filename_split[-1] = str(int(filename_split[-1]) - i)
            filename_begin = '_'.join(filename_split)
            if Path(image_root,'.'.join([filename_begin, 'webp'])).is_file():
                filename = '.'.join([filename_begin, 'webp'])
                break
            elif Path(image_root,'.'.join([filename_begin, 'jpeg'])).is_file():
                filename = '.'.join([filename_begin, 'jpeg'])
                break
            elif i==5:
                print(filename)
                filename = None
        if filename:
            fp = Path(image_root, filename)
            #ONLY get the category_id from prelabeled ones for NON overlap, overlap ones need relabeling, and shall be marked with category_id 1
            category_id = get_labeled_category(id,image_id)
            img = Image.open(fp)
            numpy_image = np.transpose(np.array(img), (1, 0, 2))
            cropped_image = crop_region(numpy_image, bounding_box)
            cropped_image = Image.fromarray(cropped_image)
            save_path = Path(cropped_path,str(id)+"-1.png")
            cropped_image.save(save_path)
            save_path = save_path.relative_to("../../../data/").as_posix()
            fp = fp.relative_to("../../../data").as_posix()
            # Fill out the data
            # image id refers to the source, non cropped image
            new_annotation = {
                "id": id,
                "category_id": category_id,
                'image_id': image_id,
                'filepath': save_path,
                'source': fp,
                "bbox": bounding_box
            }

            newfile['annotations'].append(new_annotation)
        else:
            not_found += 1

print("not_found: " + str(not_found))


not_found: 0


In [30]:
#only need the annotations entries since the rest are obsolete

not_found = 0

for entry in original["annotations"]:
    if ((entry["image_id"], entry["id"]) in overlap):
        # Only access images in the list
        id = entry["id"]
        image_id = entry['image_id']
        bounding_box = entry['bbox']
        for i in [2, 3]:
            if bounding_box[i] < 150:
                if bounding_box[i - 2] - 75 > 0:
                    bounding_box[i - 2] = bounding_box[i - 2] - 75
                    bounding_box[i] = 150
                else:
                    bounding_box[i] = 150
        filename = images_dict[image_id]['file_name']
        #print(filename)
        #get filename of file before given image
        [filename_begin, _ ] = filename.split('.')
        for i in range(1, 5):
            filename_split = filename_begin.split('_')
            filename_split[-1] = str(int(filename_split[-1]) + i)
            filename_begin = '_'.join(filename_split)
            if Path(image_root, '.'.join([filename_begin, 'webp'])).is_file():
                filename = '.'.join([filename_begin, 'webp'])
                break
            elif Path(image_root, '.'.join([filename_begin, 'jpeg'])).is_file():
                filename = '.'.join([filename_begin, 'jpeg'])
                break
            elif i == 5:
                print(filename)
                filename = None
        if filename:
            fp = Path(image_root, filename)
            img = Image.open(fp)
            numpy_image = np.transpose(np.array(img), (1, 0, 2))
            cropped_image = crop_region(numpy_image, bounding_box)
            cropped_image = Image.fromarray(cropped_image)
            save_path = Path(cropped_path, str(id) + "+1.png")
            cropped_image.save(save_path)
            save_path = save_path.relative_to("../../../data").as_posix()
            fp = fp.relative_to("../../../data").as_posix()
            # Fill out the data
            # image id refers to the source, non cropped image
            new_annotation = {
                "id": id,
                "category_id": 1,
                'image_id': image_id,
                'filepath': save_path,
                'source': fp,
                "bbox": bounding_box
            }

            newfile['annotations'].append(new_annotation)
        else:
            not_found += 1
print("not_found: " + str(not_found))


not_found: 0


In [31]:
#only need the annotations entries since the rest are obsolete

not_found = 0

for entry in original["annotations"]:
    if ((entry["image_id"], entry["id"]) in overlap):
        # Only access images in the list
        id = entry["id"]
        image_id = entry['image_id']
        bounding_box = entry['bbox']
        for i in [2, 3]:
            if bounding_box[i] < 150:
                if bounding_box[i - 2] - 75 > 0:
                    bounding_box[i - 2] = bounding_box[i - 2] - 75
                    bounding_box[i] = 150
                else:
                    bounding_box[i] = 150
        filename = images_dict[image_id]['file_name']
        #print(filename)
        #get filename of file before given image
        [filename_begin, _ ] = filename.split('.')
        for i in range(1, 5):
            filename_split = filename_begin.split('_')
            filename_split[-1] = str(int(filename_split[-1]) - i)
            filename_begin = '_'.join(filename_split)
            if Path(image_root, '.'.join([filename_begin, 'webp'])).is_file():
                filename = '.'.join([filename_begin, 'webp'])
                break
            elif Path(image_root, '.'.join([filename_begin, 'jpeg'])).is_file():
                filename = '.'.join([filename_begin, 'jpeg'])
                break
            elif i == 5:
                print(filename)
                filename = None
        if filename:
            fp = Path(image_root, filename)
            img = Image.open(fp)
            numpy_image = np.transpose(np.array(img), (1, 0, 2))
            cropped_image = crop_region(numpy_image, bounding_box)
            cropped_image = Image.fromarray(cropped_image)
            save_path = Path(cropped_path, str(id) + "-1.png")
            cropped_image.save(save_path)
            save_path = save_path.relative_to("../../../data").as_posix()
            fp = fp.relative_to("../../../data").as_posix()
            # Fill out the data
            # image id refers to the source, non cropped image
            new_annotation = {
                "id": id,
                "category_id": 1,
                'image_id': image_id,
                'filepath': save_path,
                'source': fp,
                "bbox": bounding_box
            }

            newfile['annotations'].append(new_annotation)
        else:
            not_found += 1
print("not_found: " + str(not_found))


not_found: 0


In [43]:
for entry in original["images"]:
    fp = Path(image_root,entry['file_name'])
    fp = fp.relative_to("../../../data").as_posix()
    new_images = {
        "id": entry['id'],
        "filepath": fp,
        "width": entry['width'],
        "height": entry['height']
    }
    newfile['images'].append(new_images)

In [44]:
# Converting dictionaries to json
#json_test = json.dumps(newfile)
#with open('restructured.json', 'w') as json_file:
#    json_file.write(json_test)